# Why DNAMite

This user guide will give an overview of DNAMite and its use cases.

### What is DNAMite?

DNAMite is a glass-box deep learning model that can be used for a variety of machine learning tasks. DNAMite can achieve comparable accuracy to black-box models like xgboost while being naturally interpretable. Specifically, a trained DNAMite model can be described completely by a series of individual feature and feature interaction plots (e.g. left and center), and feature importance scores can be easily obtained from these plots (e.g. right).

<img src="housing_plots.png" alt="Housing Plots" width="800" style="display: block; margin: auto;"/>

### Under the Hood

DNAMite is a Generalized Additive Model (GAM) with pairwise interactions. That is, given a set of features $X = (X_1, X_2, \ldots, X_p)$, DNAMite fits a model of the form $$ f(X) = \sum_j f_j(X_j) + \sum_{j < \ell} f_{j, \ell}(X_j, X_{\ell}).$$ Each individual feature function $f_j$ is often called a *shape function*, while each pairwise interaction function $f_{j, \ell}$ is often called an *interaction function* or *pair shape function*. DNAMite uses neural networks for each of these function, making DNAMite a Neural Additive Model (NAM) with pairwise interactions. These neural networks are learned collectively through standard neural network training techniques (mini-batch Adam).

The separable nature of additive models like DNAMite is what allows for their glass-box nature. To explain how DNAMite can very naturally be interpreted, consider a model that predicts heart failure risk from 3 features: age (X1), systolic blood pressure (X2), and cholesterol level (X3). For simplicity we will consider a model without interactions: $f(X) = f_1(X_1) + f_2(X_2) + f_3(X_3)$. First, the impact of a patient's systolic blood pressure on their heart failure risk is given directly by $f_2$, and plotting $f_2$ clearly visualizes this impact. Second, the global importance of systolic blood pressure in predicting heart failure risk is given by averaging the absolute contribution across the training data: $\frac{1}{n} \sum_i | f_2(X_2^{(i)})|$. Together, we can assess the relationship between systolic blood pressure and heart failure risk directly from the model without any extra estimation.

### What Makes DNAMite Unique?

First, DNAMite is a NAM, while the most used python packages for additive models do not use neural networks ([pygam](https://pygam.readthedocs.io/en/latest/) and [interpretml](https://interpret.ml/docs/)). While these package suffice for many applications, there are some applications which particularly benefit from the flexibility of neural networks (more on that later).

Amongst NAMs, DNAMite differs most significantly in its method of embedding features. First, DNAMite discretizes all features, including continuous features. While this is common for tree-based models, this is very uncommon for deep learning models (although it is becoming more common, e.g. [1-2]). The motivation for discretizing continuous features is that NAMs tend to learn overly smooth functions [3], and explicit discretization of continuous features allows each shape function to independently learn the optimal score in each feature bin as if the feature was categorical. After discretization, each feature bin gets a separate embedding vector that is learned during training. 

However, it turns out that such discretization actually makes shape functions too jagged, so extra regularization is necessary in order to increase smoothness. DNAMite's solution is to use a weighted sum of embeddings instead of a single embedding. The weights in the weighted sum are given by a Gaussian kernel using the distance of each bin from the inputted bin. Using a weighted sum of embeddings allows the model to utilize the ordinal information in continuous features while still using discretization. This embedding module is visualized below.

<img src="embedding_module.png" alt="Embedding Module" width="800" style="display: block; margin: auto;"/>

After incorporating kernel weighting into the embedding module, DNAMite is able to learn smoother shape functions but with more flexibility than vanilla NAMs. As an example, the below plot show on synthetic data that a standard NAM learns overly smooth shape functions, while DNAMite with kernel smoothing learns overly jagged shape function. Normal DNAMite is able to correctly balance jaggedness and smoothness and thereby best approximate the true shape function.

<img src="synthetic_plots.png" alt="Synthetic Plots" width="800" style="display: block; margin: auto;"/>

[1] Hu, Xinyu, et al. "DeeprETA: An ETA post-processing system at scale." arXiv preprint arXiv:2206.02127 (2022).

[2] Ansari, Abdul Fatir, et al. "Chronos: Learning the language of time series." arXiv preprint arXiv:2403.07815 (2024).

[3] Agarwal, Rishabh, et al. "Neural additive models: Interpretable machine learning with neural nets." Advances in neural information processing systems 34 (2021): 4699-4711.

### Why Care about Interpretability?

There's a lot of debate amongst researcher and practitioners about when interpretability is actually useful. For example, [this blog post]( https://hai.stanford.edu/news/should-ai-models-be-explainable-depends) is one example of recent work suggesting that interpretability is less important than advertised. Further, it's critical to keep in mind that model interpretations always represent correlations which may or may not be causal, so interpretable machine learning is never a replacement for causal inference. Nonetheless, we believe that using interpretable machine learning models like DNAMite is still very valuable for many reasons. 

1) **Understanding Predictions, Good and Bad**

When using a glass-box model like DNAMite, we know with 100% confidence how each feature contributes to each prediction. For cases when the model does well, this knowledge can be useful for explaining why the model is able to predict accuracy. For example, after deploying a new machine learning model, showcasing examples when the new model outperforms the previous model is more effective if it is easy to explain how/why the new model does better (e.g. the new model effectively utilizes new features). On the other hand, identifying which features cause a bad prediction can help with understanding why the model is underperforming and how it could potentially be improved.

2) **Model Auditing**

Understanding how a model is making predictions at a macro level can be crucial for validating model generalizability. For example, in one hospital network, visitation by a priest may be a strong predictor of mortality, but this feature may not be available in other hospital networks. Thus, if this feature's importance score is high, it may be worth removing the feature so that the model learns from features that are more likely to be widely available. This type of model auditing is simply not possible when using black-box models which can result in unexpected poor generalizability. Further, interpretable models can be more easily audited to look for other potentially concerns such as fairness and privacy.

3) **Making Discoveries**

While interpretable models cannot be used for identifying causal relationships, they can be used to discover patterns in the data. In this way, such models can be viewed as a very powerful tool for exploratory data analysis. If a new pattern is discovered in the data, this could be used to provoke further investigation into a potential causal relationship. 

4) **Instilling Trust**

People generally are more likely to believe a model's prediction if the model can explain how it came up with the prediction [1]. Thus, in settings where users may be weary of model predictions, an interpretable model can help instill trust that the model is not deviating significantly from existing intuition.

[1] Poursabzi-Sangdeh, Forough, et al. "Manipulating and measuring model interpretability." Proceedings of the 2021 CHI conference on human factors in computing systems. 2021.

### When Should I Use DNAMite?

For many use cases, the Microsoft [interpreml](https://interpret.ml/docs/) package has good performance and trains efficiently without a GPU. However, there are some use cases where DNAMite has functionality that interpretml lacks. We identify two such use cases here.

1) **Survival Analysis**: in survival analysis, the goal is to predict the distribution of a time-to-event random variable. Neural networks are widely used for survival analysis as they can easily produce multi-dimensional outputs useful for learning the entire time-to-event distribution. DNAMite, thus, naturally can be used for survival analysis, while tree-based models like EBM from interpretml are much harder to adapt for survival analysis. For more details see the [Survival Analysis User Guide](https://dnamite.readthedocs.io/en/latest/notebooks/user_guides/survival_analysis.html).

2) **Feature Selection**: when a dataset has many features, it makes it much harder for the model to attribute importance to any individual features. In such settings, it is desirable to learn a model that is both interpretable and feature-sparse. Since interpretml and other similar packages do not support any feature selection/sparsity, using a different model for feature selection would be required, which can lead to suboptimality. Meanwhile, DNAMite can do both feature selection and interpretable prediction. For more details see the [Feature Selection User Guide](https://dnamite.readthedocs.io/en/latest/notebooks/user_guides/feature_selection.html).